In [1]:
import tensorflow as tf
import pickle
import numpy as np
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['data_batch_2', 'data_batch_3', 'data_batch_1', 'data_batch_5', 'test_batch', 'batches.meta', 'readme.html', 'data_batch_4']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open (filename, "rb") as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1 # 归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 ## 遍历起始位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0 ,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0 
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]    
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data, batch_labels)

(50000, 3072) (50000,)
(10000, 3072) (10000,)
[[-0.45882353 -0.27843137 -0.18431373 ... -0.30980392 -0.28627451
  -0.2       ]
 [-0.37254902 -0.35686275 -0.34117647 ...  0.6627451   0.63921569
   0.6       ]
 [-0.03529412  0.03529412  0.03529412 ... -0.57647059 -0.52941176
  -0.54509804]
 ...
 [ 0.11372549  0.10588235  0.12941176 ... -0.34901961 -0.30196078
  -0.22352941]
 [ 0.10588235  0.10588235  0.11372549 ... -0.45098039 -0.45882353
  -0.46666667]
 [ 0.89019608  0.8745098   0.8745098  ...  0.31764706  0.28627451
   0.29411765]] [4 2 7 3 5 0 8 3 0 1]


In [3]:
x = tf.placeholder(tf.float32, [None, 3072])

# [None], eg: [0,6,5,3]
y = tf.placeholder(tf.int64, [None])

x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# vgg-net思想
# acc: 0.73150
conv1_1 = tf.layers.conv2d(x_image,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        activation=tf.nn.relu,
                        name='conv1_1')

conv1_2 = tf.layers.conv2d(conv1_1,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        activation=tf.nn.relu,
                        name='conv1_2')

# 16 * 16
pooling1 = tf.layers.max_pooling2d(conv1_2,
                                  (2,2), # kernel size
                                  (2,2), # stride
                                  name='pool1')

conv2_1 = tf.layers.conv2d(pooling1,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        activation=tf.nn.relu,
                        name='conv2_1')

conv2_2 = tf.layers.conv2d(conv2_1,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        activation=tf.nn.relu,
                        name='conv2_2')

# 8 * 8
pooling2 = tf.layers.max_pooling2d(conv2_2,
                                  (2,2), # kernel size
                                  (2,2), # stride
                                  name='pool2')

conv3_1 = tf.layers.conv2d(pooling2,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        activation=tf.nn.relu,
                        name='conv3_1')

conv3_2 = tf.layers.conv2d(conv3_1,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        activation=tf.nn.relu,
                        name='conv3_2')
# 4 * 4 * 32
pooling3 = tf.layers.max_pooling2d(conv3_2,
                                  (2,2), # kernel size
                                  (2,2), # stride
                                  name='pool3')

# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
# y_ -> softmax
# y -> one_hot
#loss = ylog_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,0]
correct_prediction = tf.equal(predict, y)
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps =100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if i % 100 == 0:
            print('[Train] step: %d, loss: %4.5f, acc:%4.5f'% (i, loss_val, accu_val)) 
        if i % 500 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],
                                       feed_dict={
                                           x: test_batch_data,
                                           y: test_batch_labels
                                       })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

[Train] step: 0, loss: 2.31034, acc:0.15000
(10000, 3072) (10000,)
[Test] Step: 1, acc: 0.10050
[Train] step: 100, loss: 2.20637, acc:0.25000
[Train] step: 200, loss: 1.47460, acc:0.50000
[Train] step: 300, loss: 1.62871, acc:0.55000
[Train] step: 400, loss: 1.49210, acc:0.55000
[Train] step: 500, loss: 1.21692, acc:0.50000
(10000, 3072) (10000,)
[Test] Step: 501, acc: 0.43250
[Train] step: 600, loss: 1.49948, acc:0.65000
[Train] step: 700, loss: 1.68543, acc:0.55000
[Train] step: 800, loss: 1.44355, acc:0.60000
[Train] step: 900, loss: 1.38409, acc:0.50000
[Train] step: 1000, loss: 1.04640, acc:0.75000
(10000, 3072) (10000,)
[Test] Step: 1001, acc: 0.50900
[Train] step: 1100, loss: 1.21473, acc:0.50000
[Train] step: 1200, loss: 1.38205, acc:0.60000
[Train] step: 1300, loss: 0.98548, acc:0.70000
[Train] step: 1400, loss: 1.56654, acc:0.45000
[Train] step: 1500, loss: 1.65812, acc:0.45000
(10000, 3072) (10000,)
[Test] Step: 1501, acc: 0.55650
[Train] step: 1600, loss: 1.01325, acc:0.750

In [ ]:
print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 